In [1]:
from persona_generation_prompt import PROMPT
import json

In [2]:
print(PROMPT)


You must generate a set of fictional but realistic Iranian elderly personas that represent cultural, geographical, and social diversity in Iran.

Rules:
- All personas must be elderly (age 65–90).
- Diversity must be reflected across all components, but keep the integrity and realism of each persona.
- Personas should reflect cultural and social realities of Iran.
- Reactions and attitudes do not need to be "correct" or "moral"; they may be shaped by culture, personal experience, or limitations.
- Personality traits and psychological states should be consistent with the individual’s background.
- Use the JSON format provided below.
- For every variable, use values consistent with Iranian context.
- Do not omit any variable.

Variable definitions and accepted values:

1. Biological Component (مولفه زیستی)
- General Health (سلامت عمومی): ["خوب", "متوسط", "ضعیف"]
- Chronic Disease (بیماری مزمن): [None, "فشار خون بالا", "بیماری‌های قلبی و عروقی", "دیابت نوع ۲", "آرتروز و درد مفاصل", "پوکی

In [3]:
neg_persona_rules = "- Consider this persona to have negative traits."
pos_persona_rules = "- Consider this persona to have positive traits."

def build_system_prompt(additional_rules: str) -> str:
    return PROMPT.replace("[ADDITIONAL_RULES_PLACEHOLDER]", additional_rules)

neg_persona_prompt = build_system_prompt(neg_persona_rules)
pos_persona_prompt = build_system_prompt(pos_persona_rules)

In [4]:
neg_persona_prompt

'\nYou must generate a set of fictional but realistic Iranian elderly personas that represent cultural, geographical, and social diversity in Iran.\n\nRules:\n- All personas must be elderly (age 65–90).\n- Diversity must be reflected across all components, but keep the integrity and realism of each persona.\n- Personas should reflect cultural and social realities of Iran.\n- Reactions and attitudes do not need to be "correct" or "moral"; they may be shaped by culture, personal experience, or limitations.\n- Personality traits and psychological states should be consistent with the individual’s background.\n- Use the JSON format provided below.\n- For every variable, use values consistent with Iranian context.\n- Do not omit any variable.\n\nVariable definitions and accepted values:\n\n1. Biological Component (مولفه زیستی)\n- General Health (سلامت عمومی): ["خوب", "متوسط", "ضعیف"]\n- Chronic Disease (بیماری مزمن): [None, "فشار خون بالا", "بیماری\u200cهای قلبی و عروقی", "دیابت نوع ۲", "آرت

In [5]:
from openai import OpenAI
import os

BASE_URL = "https://api.metisai.ir/openai/v1"
API_KEY = os.getenv("METIS_API_KEY")

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
)

In [6]:
model_to_use = "openai/gpt-5-nano"

TEMPERATURE = 1
TOP_P = 0.9
PRESENCE_PENALTY = 0.3
FREQUENCY_PENALTY = 0.4

In [7]:
def messages(persona_count: int):
    return [
        {
            "role": "system",
            "content": PROMPT
        },
        {
            "role": "user",
            "content": f"Generate {persona_count} personas. Only give the JSON array with no extra text or formatting. Don't wrap the array in markdown formatting."
        }
    ]


In [8]:
def call_llm(model, messages):
    resp = client.chat.completions.create(
            model=model,
            temperature=TEMPERATURE,
            # top_p=TOP_P,
            # presence_penalty=PRESENCE_PENALTY,
            # frequency_penalty=FREQUENCY_PENALTY,
            messages=messages,
        )
    return resp

def parse_response(resp):
    return resp.choices[0].message.content

In [9]:
test_messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": "What is the capital of France?"
    }
]
resp = call_llm("gpt-5-nano", test_messages)
parse_response(resp)


'Paris.'

In [10]:
resp = call_llm("gpt-4.1", messages(10))

In [11]:
personas = parse_response(resp)

In [12]:
if personas is None:
    print("Personas is None")
else:
    print(json.dumps(json.loads(personas), ensure_ascii=False, indent=4))

[
    {
        "id": 1,
        "age": 78,
        "gender": "M",
        "biological_component": {
            "general_health": "متوسط",
            "chronic_disease": "فشار خون بالا",
            "mobility": "مستقل",
            "senses": {
                "بینایی": "ضعیف",
                "شنوایی": "خوب"
            },
            "daily_energy": "متوسط"
        },
        "psychological_component": {
            "personality_type": "ISTJ",
            "cognitive_status": "فراموشی خفیف",
            "dominant_emotion": "آرام",
            "emotional_intelligence": "متوسط",
            "iq": "متوسط",
            "attitude_to_aging": "پذیرش"
        },
        "social_component": {
            "main_social_role": "پدربزرگ",
            "social_support": "خانواده پرجمعیت",
            "social_participation": "غیر فعال"
        },
        "economic_component": {
            "income": "حقوق بازنشستگی",
            "economic_decile": 5,
            "housing": "خانه شخصی"
        },
    

In [13]:
from time import time
timestamp = time()

if personas:
    with open(f"personas/personas{timestamp}.json", "w", encoding="utf-8") as f:
        f.write(json.dumps(json.loads(personas), ensure_ascii=False))
else:
    print("Personas is None")